# Volumes

You have to use `-v` option of the `docker run` command to mount volume to container. In general you have to use following syntax:


```bash
docker run \
    -v <volume1>:<path in container1> \
    -v <volume2>:<path in container2> \
    ...
```

Sometimes it's extremely useful to mount a specific folder but not a volume; you need to use folder name instead of `volume name` - this case is specific type of the volume and called **bind mount**.

## Mount to container

To mount volume to the container you have to use `-v <volume name>:<path in container>` option.

---

The following example creates a Docker container that mounts `test_volume` as `folder` and stores `my_file` with `my message`.

In [17]:
docker volume create test_volume &> /dev/null
docker run --rm -itd\
    --name test_container \
    -v test_volume:/folder alpine \
    sh -c "echo \"my message\" > /folder/my_file" &> /dev/null
sleep 3

After while that container stops and being removed. So there aren't any containers yet.

In [18]:
docker ps -a

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


But data has been saved. We can show this by mounting the same volume as the `temp` folder. If we now check the files stored in `temp` - there will be `my_file`.

In [19]:
docker run --rm -it -v test_volume:/temp alpine ls temp

my_file


And it'll store exactly the same content that we saved at the begining.

In [21]:
docker run --rm -it -v test_volume:/temp alpine cat /temp/my_file

my message


Don't forget to remove test volume after all.

In [22]:
docker volume rm test_volume

test_volume


# Bind mount

Bind mount is a specific but extremely usefull type of volume. It's a folder on the host specified by user to be mounted to the container.

---

In the following example creted folder `temp_folder`, mounted to container as  `temp_folder_inc_count` folder.

**Note** here is specified `--user $(id -u):$(id -g)` to run container with same user notebook runned. It'll garantee that commands in host will have full rights for modifying/deleting files created from container.

In [34]:
mkdir temp_folder

docker run \
    --user $(id -u):$(id -g) \
    -v $(pwd)/temp_folder:/temp_folder_in_cont \
    --rm -itd --name temp_example \
    ubuntu &> /dev/null

Now `temp_folder` on the host and `temp_folder_in_count` on the container are the same folders. So the file `container_file` created by the container will be accessible from the host and vice versa the file `host_file` created by the host will be accessible from the container.

In [31]:
docker exec temp_example \
    bash -c "echo \'hello from container\' >> temp_folder_in_cont/container_file"

echo "hello from host" >> temp_folder/host_file

Now let us check `conatiner_file` from host and `host_file` from container.

In [32]:
cat temp_folder/container_file
docker exec temp_example cat temp_folder_in_cont/host_file

'hello from container'
hello from host


Finally, don't forget to clean the system of debris.

In [35]:
docker stop temp_example &> /dev/null
rm -r temp_folder

## Read only mount

By adding `:ro` to the path of the folder in container you can make it to be read only in the container.

---

In the following example we creating file `ro_ex` which have content `original_data` and mount that file to the container.

In [43]:
echo "original data" > ro_ex
docker run --rm -idt --name ro_ex \
    -v $(pwd)/ro_ex:/experimental/ro_ex:ro \
    ubuntu &> /dev/null

Now let us try to read and modify this file from the container.

In [46]:
docker exec ro_ex cat /experimental/ro_ex
docker exec ro_ex bash -c "echo \"modified data\" > /experimental/ro_ex"

original data
bash: line 1: /experimental/ro_ex: Read-only file system


: 1

So there is no problem reading the file, but the attempt to modify it was rejected.

The next cell confirms that you can keep the file unchanged.

In [47]:
cat ro_ex

original data


Keeping your system clean.

In [48]:
docker stop ro_ex &> /dev/null
rm -r ro_ex

### Mounting `.dockerignore` files

Even if you mount the file described in .dockerignore, we will still have it in the container. 

In the following example, I create `app/ignore_file.txt` and mention it in dockerignore. Build image using this .dockerignore, but in container based on this image I mount `app` folder. And as a result I can see contents of `ignore_file.txt` regardless of what I specified in the `.dockerignore`.

In [5]:
%%bash
cd filesystem_example
mkdir app
echo "message in ignore_file.txt" > app/ignore_file.txt
echo "=====.dockerignore====="
echo "app/ignore_file.txt" > .dockerignore
cat .dockerignore
echo "=====dockerfile====="
echo "FROM ubuntu" > dockerfile
cat dockerfile

# build image with setted .dockerignore
docker build -t test_image &> /dev/null

# start container mountig file mentioned in .dockerignore
docker run --rm -itd --name ignore_ex\
    -v $(pwd)/app:/app\
    ubuntu &> /dev/null

echo "=====ignore-file from container====="
# make sure that this secret file is in the container
docker exec ignore_ex cat app/ignore_file.txt

docker stop ignore_ex &> /dev/null
docker rmi test_image &> /dev/null
rm -r app
rm .dockerignore
rm dockerfile

=====.dockerignore=====
app/ignore_file.txt
=====dockerfile=====
FROM ubuntu
=====ignore-file from container=====
message in ignore_file.txt


## Volume by default

Some containers create their own volumes by default when they run, so you may find that your entire hard drive is flooded. 

For example `yandex/clickhouse-server`. In the following cell I have some containers `yandex/clickhouse-server` running and show that each container has created a volume.

In [7]:
%%bash

echo "=====docker volume ls before====="
docker volume ls

# run clickhouse
docker run -d --name db_1 --rm yandex/clickhouse-server &> /dev/null
docker run -d --name db_2 --rm yandex/clickhouse-server &> /dev/null
docker run -d --name db_3 --rm yandex/clickhouse-server &> /dev/null

# list volumes
echo "=====docker volume ls after====="
docker volume ls
docker stop db_1 db_2 db_3 &> /dev/null

=====docker volume ls before=====
DRIVER    VOLUME NAME
=====docker volume ls after=====
DRIVER    VOLUME NAME
local     66e881449c54eb455e1a0e16b6b1cfff0aac6fbe31739ea59d307d5631c04fa2
local     b6d2ba76cf901665684319a3197166a61d8da5d75b8804945a8f0368de0c89ec
local     d3ba9de2668e8e3f68d43ccfb02db72738f2c2b7048ed2bfc94b3134d00f53f1


## Several volumes in one command

You can repeat `-v` option in `docker run` many times as you need.

Basic example.

In [9]:
%%bash
cd filesystem_example

mkdir test1 test2
echo "message in test1" > test1/my_file
echo "message in test2" > test2/my_file

docker run --rm --name example_cont -itd\
    -v $(pwd)/test1:/test1\
    -v $(pwd)/test2:/test2\
    ubuntu &> /dev/null

echo "=====files from container====="
docker exec example_cont cat test1/my_file
docker exec example_cont cat test2/my_file

docker stop example_cont &> /dev/null
rm -r test1 test2

=====files from container=====
message in test1
message in test2


But you can't mount a directory on the container twice.

In [14]:
%%bash
cd filesystem_example

mkdir test1 test2
echo "message in test1" > test1/my_file
echo "message in test2" > test2/my_file

docker run --rm --name example_cont -itd\
    -v $(pwd)/test1:/test\
    -v $(pwd)/test2:/test\
    ubuntu

rm -r test1 test2

docker: Error response from daemon: Duplicate mount point: /test.
See 'docker run --help'.


A directory from host to different folders in container is available. But you need to know that they are actually the same directory with different names - any change in one will happen in the other.

In the following example, we mount `test` in host to `test1` and `test2` in container. Then we add the file `new_file` to `test1`, but it will appear not only in `test1` but also in `test2` and `test`.

In [23]:
%%bash
cd filesystem_example

mkdir test

docker run --rm --name example_cont -itd\
    -v $(pwd)/test:/test1\
    -v $(pwd)/test:/test2\
    ubuntu &> /dev/null

docker exec example_cont bash -c "echo \"hello from container\" > test1/new_file"
echo "=====ls test1 from container====="
docker exec example_cont ls test1
echo "=====ls test2 from container====="
docker exec example_cont ls test2

docker stop example_cont &> /dev/null


echo "=====ls test====="
ls test
rm -r test

=====ls test1 from container=====
new_file
=====ls test2 from container=====
new_file
=====ls test=====
new_file
